# This a basic experimentation with SVC, Logistic Regression and Decision Tree trained and tested on the data. The tweets' texts are first converted to word vectors and then these vectors replaced the 'tweets_Text' column in the dataset so that now the features are numeric values. Also, the 'tweets_Date' column was dropped and not considered in the training set for now due to its ambigous value. Other real valued columns were normalized. 

In [68]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

In [69]:
# Replace the dataset path with your own appropiate path

data = pd.read_csv('../input/tweetsdata/Jan-Oct-2020_repliesCategorized.csv')

In [70]:
data.head(5)

,tweets_Date,tweets_Text,tweets_Replies,tweets_Retweets,tweets_Likes,has_image,has_Link,tweets_ResponseCategory
0,Sep 30,initial reports suggest three rockets struck b...,96.0,306.0,424.0,0,0,1
1,Sep 30,senate approved stopgap spending bill order av...,164.0,138.0,341.0,1,1,2
2,Sep 30,seven people shot funeral home milwaukee treat...,88.0,191.0,296.0,1,1,1
3,Sep 30,clare bronfman heiress seagram liquor fortune ...,64.0,237.0,651.0,1,1,0
4,Sep 30,trump plans participate next two presidential ...,491.0,155.0,510.0,1,0,3


In [71]:
labels = data["tweets_ResponseCategory"]
data.drop("tweets_ResponseCategory", axis=1, inplace=True)

In [72]:
cv = CountVectorizer()
vectorized_tweets = cv.fit_transform(data['tweets_Text'])
count_vect_df = pd.DataFrame(vectorized_tweets.todense(), columns=cv.get_feature_names())

new_data=pd.concat([data, count_vect_df], axis=1)


In [73]:
new_data.drop(['tweets_Text', 'tweets_Date'], axis=1, inplace=True)

In [74]:
new_data.head()

,tweets_Replies,tweets_Retweets,tweets_Likes,has_image,has_Link,017,10,100,1000,10000,...,zealands,zef,zero,zeta,zetas,zion,zone,zones,zoo,zverev
0,96.0,306.0,424.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,164.0,138.0,341.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,88.0,191.0,296.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,64.0,237.0,651.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,491.0,155.0,510.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
cols_to_norm = ['tweets_Replies', 'tweets_Retweets', 'tweets_Likes']
new_data[cols_to_norm] = preprocessing.MinMaxScaler().fit_transform(new_data[cols_to_norm])
new_data[cols_to_norm].describe()

,tweets_Replies,tweets_Retweets,tweets_Likes
count,2811.000000,2811.000000,2811.000000
mean,0.048155,0.027109,0.044302
std,0.073818,0.045918,0.068958
min,0.000000,0.000000,0.000000
25%,0.013210,0.007624,0.012749
50%,0.025099,0.014034,0.022728
75%,0.052368,0.029022,0.047824
max,1.000000,1.000000,1.000000


In [76]:
X_train, X_test, Y_train, Y_test = train_test_split(new_data, labels, test_size=0.2)

In [77]:
X_train.head(5)

,tweets_Replies,tweets_Retweets,tweets_Likes,has_image,has_Link,017,10,100,1000,10000,...,zealands,zef,zero,zeta,zetas,zion,zone,zones,zoo,zverev
848,0.156633,0.246296,0.542740,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2556,0.226269,0.073031,0.109689,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
165,0.009624,0.006627,0.008365,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1627,0.025288,0.030538,0.024719,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2586,0.086998,0.006367,0.018021,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 1. Try with Logistic Regression

In [85]:
# if max_iter is reached without convergence, we can just increase the max_iter  
solver_list = ['newton-cg', 'sag','saga','lbfgs']
score_list = []
for s in solver_list:
    clf = LogisticRegression(solver=s, max_iter=1000)
    clf.fit(X_train, Y_train)
    score_list.append(clf.score(X_test, Y_test))
print(score_list)

0.43872113676731794


## 2.Try with Decision Tree(with and without cross-validation)

In [82]:
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)

1.0

In [84]:
print(max(cross_val_score(clf, new_data, labels, cv=5)))

1.0


## 3.Try with SVM

In [67]:
clf = SVC()
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)

0.46003552397868563